In [1]:
import sys
import spotpy, pandas
import sobek, wflow

In [38]:
"""
File and folder structure suggested:
- This file as a script or as a jupyter file
- setup.ini configuration vars for coupling
- Sobek:
    - (Full model folders) default MagSec1, ...2, ...3
    - sobek.py
- Wflow
    - (Full model folders) default wflow_0, ....
    - (For single simulation) folder single
    - wflow.py
- Notes: Sobek need be running and import network click to scratch!
"""

class Couple:    
    def __init__(self):
        #Define some useful vars
        self.timeserie = pandas.date_range('2008-01-01', '2012-12-31', freq='D')
        self.lateralfile_path="Sobek/MagSect%d.lit/%d/LATERAL.DAT"
        #Start wflow calibration process
        #self.runTss = self.wflow_init()
        self.couple_init()
        
    def wflow_init(self):
        try:
            sampler = spotpy.algorithms.mc(wflow.spot_setup(), dbname='wflow_calibration', dbformat='csv')
            param = sampler.getdata.parameters()
            runTss = wflow.spot_setup.single_simulation(param)
            return runTss
        except:
            sys.exit("something wrong happened while runnig wflow!")

    def sobek_init(self):
        pass
    
    def couple_init(self):
        """
        Setup.ini filename
        This method is for setup the static files of Sobek inputs (Laterals and boundarys)
        with the output of wflow Run.tss file.
        """
        self.setup = open("setup.ini","r")
        self.setup = self.setup.readlines()
        valid1 = self.change_lateral_values(1, eval(self.setup[1])[0], eval(self.setup[1])[1])
        valid2 = self.change_lateral_values(2, eval(self.setup[2])[0], eval(self.setup[2])[1])
        valid3 = self.change_lateral_values(3, eval(self.setup[3])[0], eval(self.setup[3])[1])
        if valid1 == valid2 == valid3 == "success": pass
        else: sys.exit("something wrong happened while change the laterals!")
        #Run sobek calibration algorithm
        self.sobek_init()
        
    def change_lateral_values(self, M, V, R):
        """
        M: number of Sobek model, Int
        V: id_sobel: Number_wflow, Dictionary Python
        R: Range of V
        Case: Sobel case number, Int
        """
        self.V = V
        #Transform from run.tss to LATERAL.DAT
        self.runTss = pandas.read_table("debug/run.tss",
                                    skiprows=[i for i in range(35)], 
                                    #sep="|\s*", 
                                    #engine="python", 
                                    delim_whitespace=True,
                                    header=None)
        if M == 1:
            Case=10
            self.lateral_data = [self.runTss[:][idx] for idx in range(R[0],R[1]+1)]
            
        elif M == 2:
            Case = 4
            self.lateral_data = [self.runTss[:][idx] for idx in range(R[0],R[1]+1)]
            
        elif M == 3:
            Case = 3
            self.lateral_data = [self.runTss[:][idx] for idx in range(R[0],R[1]+1)]

        #Write the new Lateral.dat            
        with open(self.lateralfile_path%(M,Case), "wb") as f:
            for idx, values in enumerate(self.lateral_data):
                f.write("FLNO id '%s' sc 0 lt 0 dc lt 1 0 0 PDIN 0 0 '' pdin"%V[idx])
                f.write("TBLE")
                for idx2, value in enumerate(values):
                    f.write(str(self.timeserie[idx2].strftime("%Y:%m:%d:%H:%M:%S"))
                            + "\b" + str(value) + "\b" + "<\n")
                f.write("tble flno")
        return "success"

In [39]:
c = Couple()
c.couple_init()

KeyError: 33

In [14]:
setup = open("setup.ini","r")

In [54]:
setup = setup.readlines()

In [57]:
eval(setup[1])[0]

['Rio_Bache',
 'Rio_Villavieja',
 'Rio_Aipe',
 'Rio_Pata',
 'Rio_Cabrera',
 'Rio_Prado',
 'Rio_Saldana',
 'Rio_Luisa',
 'Rio_Sumapaz',
 'Rio_Bogota',
 'Rio_Coello',
 'Rio_Seco',
 'Rio_Totare',
 'Rio_Recio',
 'Rio_Lagunilla',
 'Rio_Guali',
 'Rio_Guarino']

In [22]:
timeserie = pandas.date_range('2008-01-01', '2012-12-31', freq='D')

In [24]:
i=timeserie[1]

In [31]:
i.strftime("%Y:%m:%d:%H:%M:%S")

'2008:01:02:00:00:00'